<a href="https://colab.research.google.com/github/kannanappuswamy/LLMDemo/blob/main/My_first_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implemeting a simple agent similiar to: https://til.simonwillison.net/llms/python-react-pattern

In [1]:
!pip install -U -q langchain_groq duckduckgo_search polygon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.4 MB/s eta 0:00:00


In [4]:
from langchain_groq import ChatGroq
import os
from google.colab import userdata
llm_groq = ChatGroq(model_name="llama3-70b-8192", api_key=userdata.get("GROQ_API_KEY"))
os.environ["POLYGON_API_KEY"] = userdata.get("POLYGON_API_KEY")

In [5]:
class Agent:
  def __init__(self,prompt):
    self.messages = [] # setting up a basic memory
    self.messages.append({"role": "system", "content": prompt})

  def __call__(self,user_message):
    self.messages.append({"role":"user","content":user_message})
    ai_response = llm_groq.invoke(self.messages).content
    self.messages.append({"role":"assistant","content":ai_response})
    return ai_response


In [6]:
prompt = """
You operate in a loop consisting of Thought, Action, PAUSE, and Observation.
At the end of the loop, you provide an Answer.
Use Thought to describe your reasoning about the task you have been asked to complete.
Use Action to execute one of the actions available to you, then return PAUSE.
Observation will contain the result of the action.

Your available actions are:

calculate:

e.g. calculate: result=4 * 7 / 3
Runs a calculation and returns the result. Use Python, including any necessary imports, e.g., import math; math.sqrt(4).

stock_price:

e.g. stock_price: AAPL
Returns the current stock price of the given ticker symbol (e.g., AAPL for Apple Inc.).

ddgs_news:

e.g. ddgs_news: TSLA
Performs a news search for the given ticker query using DuckDuckGo and returns a summarized text of all the news articles found. Use the news content to analyze and determine whether the overall sentiment is positive, negative, or neutral.

You are responsible for retrieving the news and performing the sentiment analysis.

Example session:

Question: What is the current price of Tesla, and is the recent news about it positive or negative?
Thought: First, I need to get the current stock price of Tesla.
Action: stock_price: TSLA
PAUSE

You will be called again with this:

Observation: The current stock price of Tesla (TSLA) is $700.

Thought: Now, I should retrieve the latest news about Tesla and determine the overall sentiment.
Action: ddgs_news: Tesla
PAUSE

You will be called again with this:

Observation: Tesla secures a new battery deal with Panasonic...\nTesla's latest earnings surpass expectations...\n\n--Next News--\n\nTesla faces regulatory challenges in Europe...\n\n--Next News--\n\n...

Thought: After analyzing the news content, the sentiment appears to be mostly positive due to the strong earnings and new battery deal.
I should calcuate the number of stocks you could buy for $10,000.

Action: calculate: result=$10000/700 = 14 shares
PAUSE

Observation: You could buy 14 shares.

Answer: The current price of Tesla (TSLA) is $700. The recent news about Tesla is generally positive, with strong earnings and a new battery deal.
You could buy 14 shares to cover your investment.


If the news is neither negative or positive and you cannot make a decisiion, return a HOLD statement. IF not give a BUY or SELL statement.
This is for a lab experiment and not a trading strategy and thus be bold.

Return the number of shares that could be bought.
""".strip()

In [7]:
from duckduckgo_search import DDGS
def search_news(query: str, region: str = 'in-en') -> str:
    try:
        news_results = DDGS().news(query, region='in-en',max_results=5)
        if not news_results:
            return f"No recent news found for {query}."

        text=""
        for article in news_results:
            text +=  article.get('title')+ "\n"+ article.get('body')+"\n\n--Next News--\n\n"

        return text

    except Exception as e:
        return f"An error occurred while searching for news: {str(e)}"

search_news("TSLA")

"Is Wall Street Underestimating Tesla (TSLA) Stock?\nToday shares of Tesla have fallen -8.4%, vindicating the analysts who have given the stock an average rating of hold. But could they be wrong? Some factors show that Tesla may actually be undervalued at today's prices,\n\n--Next News--\n\nWall Street sets Tesla (TSLA) stock price for next 12 months\nAs Tesla announced FSD release for Europe and China, Wall Street analysts reacted by adjusting TSLA price targets for the next 12 months.\n\n--Next News--\n\nTesla Inc (TSLA) Stock: Greater Than Its Current Valuation?\nInvestors often prefer to invest in shares held by company insiders, as management is typically more aligned with the company's success and shareholder interests. Currently, insiders own just 12.98% of Tesla Inc (TSLA)'s shares, while financial institutions hold 46.15%.\n\n--Next News--\n\nTesla, Inc. (TSLA): One of the Best Battery Stocks to Buy According to Short Sellers\nWe recently compiled a list of the 10 Best Battery

In [8]:
import polygon
stocks_client = polygon.StocksClient(os.getenv("POLYGON_API_KEY"))
def get_stock_price(ticker: str)->str:
    return stocks_client.get_previous_close(ticker)["results"][0]["c"]

get_stock_price("AAPL")

220.82

In [9]:
def calculate(expression: str) -> str:
    """
    Executes a mathematical expression and returns the result.
    The expression can include imports and should store the result in a variable named 'result'.
    """
    print(expression)  # Debugging: print the expression to see what is being executed
    try:
        # Prepare a local namespace to safely execute the expression
        local_namespace = {}

        # Execute the full expression in the given local namespace
        exec(expression, {}, local_namespace)

        # Retrieve the result from the local namespace
        return str(local_namespace.get('result', "No result found"))

    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Example usage
print(calculate("import math; result = math.sqrt(4) * math.log(10)"))

import math; result = math.sqrt(4) * math.log(10)
4.605170185988092


In [10]:
calculate("result = 10000/27")

result = 10000/27


'370.3703703703704'

In [11]:
known_actions = {
    "ddgs_news": search_news,
    "stock_price": get_stock_price,
    "calculate": calculate
}

In [12]:
def simple_query(question):
  bot = Agent(prompt)
  return bot(question)

response = simple_query("Is Tesla a good buy?")
response

'Thought: To determine if Tesla is a good buy, I need to consider the current stock price and the recent news about the company.\n\nAction: stock_price: TSLA\nPAUSE'

In [13]:
import re
def parse_actions(ai_message):
  action_re = re.compile('^Action: (\w+): (.*)$')
  actions = [action_re.match(a) for a in ai_message.split('\n') if action_re.match(a)]
  print(actions)

  if actions:
        action, action_input = actions[0].groups()
        if action not in known_actions:
              raise Exception("Unknown action: {}: {}".format(action, action_input))
        print(" -- running {} {}".format(action, action_input))
        observation = known_actions[action](str(action_input))
        return observation

  return None

parse_actions(response)

[<re.Match object; span=(0, 25), match='Action: stock_price: TSLA'>]
 -- running stock_price TSLA


210.73

In [14]:
calculate("$10000/214.11")

$10000/214.11


'Error in calculation: invalid syntax (<string>, line 1)'

In [15]:
def multi_step_reasoning(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

        # Check if the result contains an 'Answer' (indicating the final step)
        if "Answer:" in result:
            print("Final Answer:", result)
            break

        observation = parse_actions(result)

        if observation is None:
            break
        print("Observation:", observation)
        next_prompt = "Observation: {}".format(observation)

# Example usage
multi_step_reasoning("Is Tesla a good buy? How many can I buy with my pension fund of $50000")

Thought: To determine if Tesla is a good buy, I need to consider the current stock price and the sentiment of the recent news about the company.

Action: stock_price: TSLA
PAUSE

(Waiting for observation...)

Please provide the observation.
[<re.Match object; span=(0, 25), match='Action: stock_price: TSLA'>]
 -- running stock_price TSLA
Observation: 210.73
Thought: Now that I have the current stock price of Tesla, I should retrieve the latest news about Tesla and determine the overall sentiment.

Action: ddgs_news: Tesla
PAUSE
[<re.Match object; span=(0, 24), match='Action: ddgs_news: Tesla'>]
 -- running ddgs_news Tesla
Observation: The Tesla classic - and the owners who cherish Elon Musk's first electric car
Tesla's introduction to the UK came not via the 2014 Model S, but the 2008 Roadster - produced with some serious help from Lotus in Norfolk

--Next News--

Fire erupts at Elon Musk's Tesla service-showroom in Nepal
A fire broke out at Tesla's service showroom in Nepal's Kathmandu

In [16]:
print(multi_step_reasoning("Compare Microsoft and Apple and see which one I could buy for my portfolio now",10))

Thought: To compare Microsoft and Apple, I need to get the current stock prices of both companies.

Action: stock_price: MSFT
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: MSFT'>]
 -- running stock_price MSFT
Observation: 401.7
Thought: I have the current stock price of Microsoft. Now, I need to get the current stock price of Apple.

Action: stock_price: AAPL
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: AAPL'>]
 -- running stock_price AAPL
Observation: 220.82
Thought: Now that I have the current stock prices of both Microsoft and Apple, I should retrieve the latest news about each company and determine the overall sentiment for each.

Action: ddgs_news: MSFT
PAUSE
[<re.Match object; span=(0, 23), match='Action: ddgs_news: MSFT'>]
 -- running ddgs_news MSFT
Observation: Insider Sale: CEO Satya Nadella Sells 78,353 Shares of Microsoft Corp (MSFT)
Satya Nadella, CEO of Microsoft Corp (NASDAQ:MSFT), executed a significant transaction by selling 78